In [1]:
%pylab inline
from math import inf

Populating the interactive namespace from numpy and matplotlib


# Loading dataset

In [2]:
from sklearn.datasets import load_boston

In [3]:
X, y = load_boston(return_X_y=True)
print("total {} objects".format(len(X)))

total 506 objects


In [4]:
bound = round(0.75 * len(X))
X_train, y_train = X[:bound], y[:bound]
X_test, y_test = X[bound:], y[bound:]

# Helper functions

In [5]:
def H(y):
    p = {}
    for i in y:
        if not i in p:
            p[i] = 0
        p[i] += 1. / len(y)
    res = 0
    for s in p.values():
        res += s * (1 - s)
    return res

In [6]:
def split_score(X, y, k, bound):
    lower = []
    upper = []
    for i in range(len(y)):
        if X[i][k] <= bound:
            lower += [y[i]]
        else:
            upper += [y[i]]
    return H(lower) * len(lower) / len(X) + H(upper) * len(upper) / len(X)

# Recursive tree building function

In [7]:
def build_tree(X, y, max_depth = inf):
    X = np.array(X)
    y = np.array(y)
    n = len(X)
    m = len(X[0])
    leaf = {'l': None, 'r': None, 'result': np.mean(y)}
    if n == 1 or max_depth == 0:
        return leaf
    node = {'score': inf}
    for k in range(m):
        vals = sorted([X[i][k] for i in range(n)])
        bounds = [(vals[j] + vals[j + 1]) * 0.5 for j in range(n - 1)]
        for bound in bounds:
            cur = split_score(X, y, k, bound)
            if cur < node['score']:
                node['score'] = cur
                node['id'] = k
                node['bound'] = bound
    Xl = []
    Xr = []
    yl = []
    yr = []
    for i in range(n):
        if X[i][node['id']] <= node['bound']:
            Xl += [X[i]]
            yl += [y[i]]
        else:
            Xr += [X[i]]
            yr += [y[i]]
    if len(Xl) == 0 or len(Xr) == 0:
        return leaf
    node['l'] = build_tree(Xl, yl, max_depth=max_depth-1)
    node['r'] = build_tree(Xr, yr, max_depth=max_depth-1)
    return node

In [8]:
print(build_tree(X, y, max_depth=1))

{'score': 0.9833316840040305, 'id': 5, 'bound': 7.7835000000000001, 'l': {'l': None, 'r': None, 'result': 21.524329896907215}, 'r': {'l': None, 'r': None, 'result': 45.823809523809523}}


# Recursive predict function

In [9]:
def predict_value(x, node):
    if 'result' in node:
        return node['result']
    if x[node['id']] <= node['bound']:
        return predict_value(x, node['l'])
    else:
        return predict_value(x, node['r'])

# Main class

In [10]:
class DecisionTree:
    def __init__(self, max_depth=inf):
        self.max_depth = max_depth
        
    def fit(self, X, y):
        self.root = build_tree(X, y, max_depth=self.max_depth)
        return self
    
    def predict(self, X):
        return [predict_value(x, self.root) for x in X]

# Testing tree on dataset and comparing with sklearn decision tree

In [11]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

In [12]:
def run(depth):
    t = DecisionTree(max_depth=depth)
    t2 = DecisionTreeRegressor(max_depth=depth)
    t.fit(X_train, y_train)
    t2.fit(X_train, y_train)
    y_pred = t.predict(X_test)
    y_pred_2 = t2.predict(X_test)
    print("MSE score of DecisionTree is ", mean_squared_error(y_test, y_pred))
    print("MSE score of sklearn tree is ", mean_squared_error(y_test, y_pred_2))

In [13]:
for d in [1, 3, 5, 10, 100, 100000]:
    print("max_depth", d)
    run(d)

max_depth 1
MSE score of DecisionTree is  107.028874075
MSE score of sklearn tree is  89.2651884691
max_depth 3
MSE score of DecisionTree is  93.0232812596
MSE score of sklearn tree is  31.3832155653
max_depth 5
MSE score of DecisionTree is  71.5451355301
MSE score of sklearn tree is  23.8470745162
max_depth 10
MSE score of DecisionTree is  45.6787061379
MSE score of sklearn tree is  21.4484129625
max_depth 100
MSE score of DecisionTree is  45.5386507937
MSE score of sklearn tree is  41.8293650794
max_depth 100000
MSE score of DecisionTree is  45.5386507937
MSE score of sklearn tree is  37.174047619
